## SQL Demo Code

### Imports
For the necessary libraries to be available, make sure you have run in your terminal:
* `pip install requests`
* `pip install bs4`
* `pip install mysql-connector-python`

And run the cell below to `import` those libraries into this notebook!

In [1]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

### BeautifulSoup Code and SQL Connections
To connect to SQL, we can create variables called `cnx` and `cursor`, as shown below.

In [3]:
cnx = mysql.connector.connect(user = 'wsa',
                              host = '34.68.250.121',
                              database = 'Tutorials-Fall2024',
                              password = 'LeBron>MJ!')
cursor = cnx.cursor(buffered = True)

The next three cells contain the code we went over last week to scrape and save Michigan Football Gamelog Data.

In [7]:
all_rows = []
game_stats = []
start_year = 2011
end_year = 2024

def get_rows(year):
    url = requests.get(f'https://www.sports-reference.com/cfb/schools/michigan/{year}/gamelog/')
    soup = BeautifulSoup(url.text, 'html.parser')
    table = soup.find('div', attrs = {'id': 'div_offense'}).find('table')
    year_rows = table.find('tbody').find_all('tr')
    for row in year_rows:
        all_rows.append(row)

def get_stats(row):
    columns = row.find_all('td')
    date = columns[0].find('a').text
    year = int(date.split('-')[0])

    # To make sure bowl games are counted in correct season
    if date.split('-')[1] == '01':
        year -= 1

    location = columns[1].text
    if location == '':
        site = 'Home'
    elif location == '@':
        site = 'Away'
    else:
        # location == 'N'
        site = 'Neutral'

    opponent = columns[2].find('a').text

    # We are getting multiple data points of interest from this single column, so we must split it
    result_list = columns[3].text.split(' ')
    result = result_list[0]
    points_scored = int(result_list[1].split('-')[0][1:])
    points_against = int(result_list[1].split('-')[1][:-1])

    pass_cmp = float(columns[4].text)
    pass_att = int(columns[5].text)
    pass_pct = float(columns[6].text)
    pass_yrds = int(columns[7].text)
    pass_td = int(columns[8].text)
    pass_1st_down = int(columns[16].text)

    rush_att = int(columns[9].text)
    rush_yrds = int(columns[10].text)
    rush_td = int(columns[12].text)
    rush_1st_down = int(columns[17].text)

    total_offense = int(columns[14].text)
    fumbles = int(columns[22].text)
    ints = int(columns[23].text)

    return([date, year, opponent, site, result, points_scored, points_against, pass_cmp, pass_att, pass_pct,
             pass_yrds, pass_td, pass_1st_down, rush_att, rush_yrds, rush_td, rush_1st_down, total_offense, fumbles, ints])


In [9]:
for year in range(start_year, end_year+1):
    get_rows(str(year))

for row in all_rows:
    game_stats.append(get_stats(row))

In [10]:
for values in game_stats:
    print(values)

['2011-09-03', 2011, 'Western Michigan', 'Home', 'W', 34, 10, 9.0, 13, 69.2, 98, 0, 5, 26, 190, 3, 9, 288, 0, 0]
['2011-09-10', 2011, 'Notre Dame', 'Home', 'W', 35, 31, 11.0, 24, 45.8, 338, 4, 10, 26, 114, 1, 5, 452, 0, 3]
['2011-09-17', 2011, 'Eastern Michigan', 'Home', 'W', 31, 3, 7.0, 18, 38.9, 95, 2, 5, 50, 376, 2, 19, 471, 0, 1]
['2011-09-24', 2011, 'San Diego State', 'Home', 'W', 28, 7, 8.0, 17, 47.1, 93, 0, 3, 45, 320, 4, 14, 413, 2, 2]
['2011-10-01', 2011, 'Minnesota', 'Home', 'W', 58, 0, 18.0, 25, 72.0, 217, 3, 10, 48, 363, 3, 19, 580, 0, 0]
['2011-10-08', 2011, 'Northwestern', 'Away', 'W', 42, 24, 19.0, 28, 67.9, 362, 2, 14, 50, 179, 4, 8, 541, 0, 3]
['2011-10-15', 2011, 'Michigan State', 'Away', 'L', 14, 28, 12.0, 31, 38.7, 168, 1, 6, 36, 82, 1, 8, 250, 0, 1]
['2011-10-29', 2011, 'Purdue', 'Home', 'W', 36, 14, 10.0, 17, 58.8, 196, 0, 7, 53, 339, 4, 17, 535, 0, 2]
['2011-11-05', 2011, 'Iowa', 'Away', 'L', 16, 24, 18.0, 38, 47.4, 196, 2, 10, 37, 127, 0, 10, 323, 1, 1]
['2011-1

### Create a SQL Table
Before uploading our data, we first need to create the table in the correct schema in MySQL or Sequel Ace. Then, if the data looks good, we can uncomment the lines below and add the data to our table.

IMPORTANT: When you are inserting data into the WSA database, make sure you are using the *Tutorials-Fall2024* schema and your own name for your table.

In [ ]:
# When creating the statement line, you need one instance of `%s` in the VALUES for each column of data
# Here, we are inserting 20 values into each row

# Change This!
table_name = "Michigan_GameLogs_table_name"

statement = (f"INSERT INTO {table_name} "
             "(date, year, opponent, home_away, result, points_scored, points_against, pass_cmp, pass_att, pass_pct,"
             " pass_yrds, pass_td, pass_1st_down, rush_att, rush_yrds, rush_td, rush_1st_down, total_offense, fumbles, ints) "
             "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

for values in game_stats:
    cursor.execute(statement, values)
    cnx.commit()

### Example SQL Commands

1. <code>SELECT * FROM \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name;</code>
    * Used to select all of our rows from the table
2. <code>SELECT opponent, result FROM \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name;</code>
    * Used to select only certain columns from the table
3. <code>SELECT opponent, result, points_scored FROM \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name WHERE points_scored >= 35;</code>
    * Used to select only certain columns from the table when a certain condition is met
4. <code>SELECT * FROM \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name ORDER BY pass_yrds DESC;</code>
    * Used to order our table by one of the columns
5. <code>INSERT INTO \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name (year, opponent, result, points_scored, points_against) VALUES (2024, 'Ohio State', 'W', 42, 0);</code>
    * Used to insert a new row into the bottom of our table
    * If you run example 1 again, the new game will be visible at the bottom
6. <code>UPDATE \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name SET points_scored = 56 WHERE id = [insert correct id]</code>
    * Used to update a row in the table using its id
7. <code>DELETE FROM \`Tutorials-Fall2024\`.Michigan_GameLogs_your_name WHERE id = [insert correct id];</code>
    * Used to delete a row from the table using its id